In [1]:
from google.colab import drive
# Monta o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## extrair CPU power

In [6]:
import os
import pandas as pd


def extract_cpu_power(log_lines):
    power_values = []
    for line in log_lines:
        if "Total CPU Power" in line:
            parts = line.split("Total CPU Power : ")
            if len(parts) > 1:
                power_value = parts[1].split(" W")[0]
                power_values.append(power_value)
    return power_values

def process_directory(directory_path):

    # Dicionário para armazenar os dados de cada subdiretório
    data_dict = {}

    # Usa os.walk para percorrer todos os subdiretórios
    for root, dirs, files in os.walk(directory_path):
        # Verifica se há arquivos codeCarbon_time_log ou codeCarbon_cypress_log no diretório atual
        cypress_log = None
        time_log = None

        for filename in files:
            if filename.startswith('codeCarbon_cypress_log'):
                cypress_log = os.path.join(root, filename)
            elif filename.startswith('codeCarbon_time_log'):
                time_log = os.path.join(root, filename)

        # Se ambos os arquivos existirem no diretório atual, processa-os
        if cypress_log and time_log:
            # Extrai os valores de power dos arquivos
            with open(cypress_log, "r") as file:
                cypress_power = extract_cpu_power(file.readlines())
            with open(time_log, "r") as file:
                time_power = extract_cpu_power(file.readlines())

            # Cria um DataFrame para o subdiretório atual
            df = pd.DataFrame({
                "Total CPU Power": ["Total CPU Power :"] * max(len(cypress_power), len(time_power)),
                filename: cypress_power + [""] * (max(len(cypress_power), len(time_power)) - len(cypress_power)),
                "Sem rodar nada": time_power + [""] * (max(len(cypress_power), len(time_power)) - len(time_power))
            })

            # Armazena o DataFrame no dicionário, usando o nome do subdiretório como chave
            subdir_name = os.path.basename(root)
            data_dict[subdir_name] = df

    # Salva cada DataFrame em uma planilha separada
    for subdir_name, df in data_dict.items():
        output_path = os.path.join(directory_path, subdir_name,f"{subdir_name}_power_results.csv")
        df.to_csv(output_path, index=False)
        print(f"Planilha salva em: {output_path}")

if __name__ == "__main__":
    # Substitua pelo caminho do diretório no Google Drive
    directory_path = "/content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos"
    process_directory(directory_path)

Planilha salva em: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/LoginCreateAndDeleteComment/LoginCreateAndDeleteComment_power_results.csv
Planilha salva em: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/updateProfile/updateProfile_power_results.csv
Planilha salva em: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/singUpAndLogin/singUpAndLogin_power_results.csv
Planilha salva em: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/CreateComment/CreateComment_power_results.csv
Planilha salva em: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/createArticle/createArticle_power_results.csv
Planilha salva em: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/updateArticle/updateArticle_power_results.csv
Planilha salva em: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/s

## extrair outros dados:


```
Emissões de carbono em kg,Tempo em segundos (10x),Consumo total kWh,emissão de CO2eq/s,estimativa de kg.CO2eq/year
```


In [5]:
## extrair outros dados com média e mediana de CPU power

import os
import pandas as pd
import numpy as np
from google.colab import drive

# Função para extrair os dados do arquivo de log
def extract_log_data(file_path):
    with open(file_path, "r") as file:
        log_lines = file.readlines()

    data = {
        "diretório": None,  # Nova coluna para o diretório
        "emissões_kg": None,
        "tempo_segundos": None,
        "energia_kWh": None,
        "co2eq_por_segundo": None,
        "co2eq_por_ano": None,
        "media_cpu_power": None,  # Nova coluna para média de CPU power
        "mediana_cpu_power": None  # Nova coluna para mediana de CPU power
    }

    # Variáveis temporárias para armazenar os valores
    energia_kWh = None
    co2eq_por_segundo = None
    co2eq_por_ano = None

    for line in log_lines:
        if "Emissões de carbono:" in line:
            data["emissões_kg"] = float(line.split("Emissões de carbono: ")[1].split(" kg")[0])
        elif "Tempo de execução:" in line:
            data["tempo_segundos"] = float(line.split("Tempo de execução: ")[1].split(" segundos")[0])
        elif "kWh of electricity used since the beginning." in line:
            # Extrai o valor de energia_kWh
            energia_kWh = float(line.split("INFO     ")[1].split(" kWh")[0])
        elif "g.CO2eq/s mean an estimation of" in line:
            # Extrai os valores de co2eq_por_segundo e co2eq_por_ano
            relevant_part = line.split("INFO     ")[1].strip()
            print(f"Linha processada: {relevant_part}")  # Depuração

            # Usa expressão regular para extrair os valores de g.CO2eq/s e kg.CO2eq/year
            import re
            match = re.search(r"([\d.]+) g\.CO2eq/s mean an estimation of ([\d.]+) kg\.CO2eq/year", relevant_part)
            if match:
                co2eq_por_segundo = float(match.group(1))
                co2eq_por_ano = float(match.group(2))
            else:
                print(f"Aviso: Formato inesperado para kg.CO2eq/year no arquivo {file_path}")

    # Atribui os valores da última ocorrência
    data["energia_kWh"] = energia_kWh
    data["co2eq_por_segundo"] = co2eq_por_segundo
    data["co2eq_por_ano"] = co2eq_por_ano

    return data

# Função para extrair dados de CPU power dos arquivos gerados pelo primeiro script
def get_cpu_power_stats(directory_path, subdir_name):
    power_file = os.path.join(directory_path, subdir_name, f"{subdir_name}_power_results.csv")
    if os.path.exists(power_file):
        try:
            df_power = pd.read_csv(power_file)
            # Pega a coluna que contém os valores de power (ignorando a primeira coluna e a coluna "Sem rodar nada")
            power_col = [col for col in df_power.columns if col not in ["Total CPU Power", "Sem rodar nada"]][0]

            # Converte valores para numérico, ignorando strings vazias
            power_values = pd.to_numeric(df_power[power_col].replace("", np.nan), errors='coerce').dropna()

            if not power_values.empty:
                return {
                    "media": power_values.mean(),
                    "mediana": power_values.median()
                }
        except Exception as e:
            print(f"Erro ao processar arquivo de power {power_file}: {e}")
    return {"media": None, "mediana": None}

# Função para processar o diretório e consolidar os dados
def process_directory(directory_path):
    # Lista para armazenar os dados de todos os arquivos
    all_data = []

    # Percorre todos os subdiretórios
    for root, dirs, files in os.walk(directory_path):
        for filename in files:
            if filename.startswith("codeCarbon_cypress_log"):
                file_path = os.path.join(root, filename)
                print(f"Processando arquivo: {file_path}")

                # Extrai os dados do arquivo
                data = extract_log_data(file_path)

                # Adiciona o diretório relativo aos dados
                subdir_name = os.path.relpath(root, directory_path)
                data["diretório"] = subdir_name

                # Obtém estatísticas de CPU power
                power_stats = get_cpu_power_stats(directory_path, subdir_name)
                data["media_cpu_power"] = power_stats["media"]
                data["mediana_cpu_power"] = power_stats["mediana"]

                # Adiciona os dados à lista
                all_data.append(data)

    # Cria um DataFrame com os dados consolidados
    df = pd.DataFrame(all_data, columns=[
        "diretório", "emissões_kg", "tempo_segundos", "energia_kWh",
        "co2eq_por_segundo", "co2eq_por_ano", "media_cpu_power", "mediana_cpu_power"
    ])

    # Salva o DataFrame em uma planilha
    output_path = os.path.join(directory_path, "consolidated_results.csv")
    df.to_csv(output_path, index=False)
    print(f"Planilha consolidada salva em: {output_path}")

# Substitua pelo caminho do diretório no Google Drive
directory_path = "/content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos"
process_directory(directory_path)

Processando arquivo: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/LoginCreateAndDeleteComment/codeCarbon_cypress_log_20250326_204902.txt
Linha processada: 0.000698 g.CO2eq/s mean an estimation of 22.013166062613294 kg.CO2eq/year
Linha processada: 0.000730 g.CO2eq/s mean an estimation of 23.005793849174474 kg.CO2eq/year
Linha processada: 0.000735 g.CO2eq/s mean an estimation of 23.193968433463564 kg.CO2eq/year
Linha processada: 0.000718 g.CO2eq/s mean an estimation of 22.653425203961476 kg.CO2eq/year
Linha processada: 0.000716 g.CO2eq/s mean an estimation of 22.590230312604355 kg.CO2eq/year
Linha processada: 0.000734 g.CO2eq/s mean an estimation of 23.142838114963446 kg.CO2eq/year
Linha processada: 0.000713 g.CO2eq/s mean an estimation of 22.47534666807339 kg.CO2eq/year
Processando arquivo: /content/drive/MyDrive/BCC/TCC II/Medições/React + Ruby on Rails/Experimentos/updateProfile/codeCarbon_cypress_log_20250210_224815.txt
Linha processada: 0.000669 